In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="API_KEY_HERE")
project = rf.workspace("uni-o612z").project("facial-emotion-recognition")
version = project.version(5)
dataset = version.download("yolov11")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 120.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Facial-emotion-recognition-5 in yolov11:: 100%|██████████| 21738/21738 [00:09<00:00, 2413.40it/s]


# Task
Install `ultralytics`, then configure the `data.yaml` file within the `/content/Facial-emotion-recognition-5` directory to correctly point to the image and label paths, and finally, train a YOLO model on the facial emotion recognition dataset.

## Install Ultralytics and Dependencies

### Subtask:
Install the `ultralytics` library, which provides an easy-to-use interface for training and testing various YOLO models, and any other necessary dependencies for the environment.


In [5]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.9 MB/s eta 0:00:00


## Prepare Data Configuration

### Subtask:
Ensure the downloaded dataset's `data.yaml` file is correctly configured to point to the training, validation, and testing image and label paths within the `/content/Facial-emotion-recognition-5` directory.


**Reasoning**:
The first step in configuring the `data.yaml` file is to locate it within the specified directory. Listing the contents of `/content/Facial-emotion-recognition-5` will help confirm its presence and the directory structure.



In [6]:
import os

dataset_path = '/content/Facial-emotion-recognition-5'

# List the contents of the dataset directory to locate data.yaml and understand the structure
print(f"Listing contents of {dataset_path}:")
!ls -F {dataset_path}

# Also list the contents of the train, val, and test directories to understand image/label paths
print(f"\nListing contents of {dataset_path}/train:")
!ls -F {dataset_path}/train

print(f"\nListing contents of {dataset_path}/valid:")
!ls -F {dataset_path}/valid

print(f"\nListing contents of {dataset_path}/test:")
!ls -F {dataset_path}/test

Listing contents of /content/Facial-emotion-recognition-5:
data.yaml  README.dataset.txt  README.roboflow.txt  test/  train/  valid/

Listing contents of /content/Facial-emotion-recognition-5/train:
images/  labels/

Listing contents of /content/Facial-emotion-recognition-5/valid:
images/  labels/

Listing contents of /content/Facial-emotion-recognition-5/test:
images/  labels/


**Reasoning**:
To correctly configure the `data.yaml` file, I need to read its current content to identify the paths for training, validation, and testing images and labels, as well as the number and names of classes.



In [7]:
data_yaml_path = os.path.join(dataset_path, 'data.yaml')

print(f"Reading content of {data_yaml_path}:")
with open(data_yaml_path, 'r') as f:
    print(f.read())

Reading content of /content/Facial-emotion-recognition-5/data.yaml:
train: ../train/images
val: ../valid/images
test: ../test/images

nc: 4
names: ['angry', 'happy', 'sad', 'surprised']

roboflow:
  workspace: uni-o612z
  project: facial-emotion-recognition
  version: 5
  license: CC BY 4.0
  url: https://universe.roboflow.com/uni-o612z/facial-emotion-recognition/dataset/5


**Reasoning**:
The current `data.yaml` uses incorrect relative paths for the `train`, `val`, and `test` image directories. Based on the directory structure observed previously, I need to update these paths to correctly reflect their location within the `Facial-emotion-recognition-5` directory.



In [8]:
import yaml

# Read the existing content of data.yaml
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

# Update the paths to be relative to the dataset_path
# The train, val, test directories are directly inside the dataset_path
data_config['train'] = os.path.join('train', 'images')
data_config['val'] = os.path.join('valid', 'images')
data_config['test'] = os.path.join('test', 'images')

# Save the updated content back to data.yaml
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"Updated content of {data_yaml_path}:")
with open(data_yaml_path, 'r') as f:
    print(f.read())

Updated content of /content/Facial-emotion-recognition-5/data.yaml:
names:
- angry
- happy
- sad
- surprised
nc: 4
roboflow:
  license: CC BY 4.0
  project: facial-emotion-recognition
  url: https://universe.roboflow.com/uni-o612z/facial-emotion-recognition/dataset/5
  version: 5
  workspace: uni-o612z
test: test/images
train: train/images
val: valid/images



## Train YOLO Model

### Subtask:
Configure and initiate the training process for a YOLO model (e.g., YOLOv8) using the `ultralytics` library and the prepared dataset. This will involve setting training parameters like epochs and batch size.


**Reasoning**:
To begin the training process, I will import the `YOLO` model from the `ultralytics` library, load a pre-trained `yolov8n` model, and then initiate training using the `data.yaml` file, setting the number of epochs to 50 and image size to 640.



In [9]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

# Train the model with the configured data.yaml, 130 epochs, and image size 540
results = model.train(data=data_yaml_path, epochs=130, imgsz=540)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Facial-emotion-recognition-5/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.5, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=Fals

## Evaluate YOLO Model

### Subtask:
After training, evaluate the model's performance on the validation or test set using the `ultralytics` library to calculate key metrics such as mean Average Precision (mAP), precision, and recall.


**Reasoning**:
The training of the YOLO model has started, but it's an ongoing process. To evaluate the model's performance as per the subtask, I need to add a step to call the `val()` method on the trained `model` object. This method will compute key metrics like mAP, precision, and recall on the validation set after the training is complete.



In [10]:
metrics = model.val()

print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")
print(f"Precision: {metrics.box.p}")
print(f"Recall: {metrics.box.r}")

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1202.2±930.5 MB/s, size: 109.9 KB)
val: Scanning /content/Facial-emotion-recognition-5/valid/labels.cache... 892 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 892/892 1.4Mit/s 0.0s
val: /content/Facial-emotion-recognition-5/valid/images/images-2020-11-06T004041-046_face_png.rf.356e9608a7ae873d0819895236165983.jpg: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 56/56 6.1it/s 9.1s
                   all        892        948      0.731      0.724      0.731      0.449
                 angry        207        213      0.766      0.693      0.815      0.699
                 happy        246        283      0.709       0.82      0.736      0.336
                   sad        232        233